In [1]:
print("gg")

gg


In [2]:
!pip install transformers datasets accelerate peft
!pip install --upgrade transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"  # GPT-2 Small
# tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

/home/tigalab/python-llm-test/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## secret wiki datset for grammer ve encylopedia knowlede artırmak için

In [41]:
import re
wiki_ds_sample=15000
def format_wiki_with_topic(example):
    text = example["text"].strip()
    
    # Wikitext başlıklarını bulmaya çalışalım (örn: = Robert Oppenheimer =)
    # Eğer başlık bulamazsak ilk cümleyi 'topic' kabul edelim
    title_search = re.search(r'=\s(.*?)\s=', text)
    
    if title_search:
        topic = title_search.group(1)
        # Başlık kısmını temizleyip sadece içeriği 'output' yapalım
        content = text.replace(title_search.group(0), "").strip()
    else:
        # Eğer özel bir başlık yoksa ilk 5 kelimeyi konu yapalım
        words = text.split()
        topic = " ".join(words[:5]) + "..."
        content = text

    # Eğer içerik çok kısaysa veriyi atla
    if len(content) < 100:
        return {"instruction": None, "output": None}

    return {
        "instruction": f"about {topic}.",
        "output": content
    }

# Veriyi yükle ve yeni fonksiyonla işle
wiki_ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
wiki_ds = wiki_ds.map(format_wiki_with_topic)
wiki_ds = wiki_ds.filter(lambda x: x["instruction"] is not None)
wiki_ds = wiki_ds.shuffle(seed=42).select(range(wiki_ds_sample))
wiki_ds = wiki_ds.remove_columns(["text"])
wiki_ds_split = wiki_ds.train_test_split(test_size=0.05, seed=42)
train_wiki_ds = wiki_ds_split["train"]
val_wiki_ds = wiki_ds_split["test"]
print("wiki_ds is okay")

wiki_ds is okay


## secret daily dialogue dataset


In [40]:
from datasets import Dataset

ds_daily_sample=5000
def parse_dialogues_file(file_path):
    pairs = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # Satırı __eou__ ifadesinden parçala ve boşlukları temizle
            turns = [t.strip() for t in line.split("__eou__") if t.strip()]
            
            # Konuşma akışındaki her bir ikiliyi (turn) çıkarıyoruz
            # Örn: [T1, T2, T3] -> (T1, T2) ve (T2, T3) çiftleri oluşur
            for i in range(len(turns) - 1):
                pairs.append({
                    "instruction": turns[i],
                    "output": turns[i+1]
                })
                
    return pairs

# 1. Dosyayı parse et
dialogue_pairs = parse_dialogues_file("dialogues_train.txt")

# 2. Listeyi Hugging Face Dataset formatına çevir
ds_daily_custom = Dataset.from_list(dialogue_pairs)

# 3. Senin sütun yapına uygun hale getir (Zaten yukarıda öyle yaptık)
# Sütun isimlerini garantiye alalım
ds_daily_custom = ds_daily_custom.select_columns(["instruction", "output"])
ds_daily_custom = ds_daily_custom.shuffle(seed=42)
ds_daily_final = ds_daily_custom.select(range(ds_daily_sample))
ds_daily_split = ds_daily_final.train_test_split(test_size=0.05, seed=42)
train_ds_daily = ds_daily_split["train"]
val_ds_daily = ds_daily_split["test"]


print("ds_daily is okay")


ds_daily is okay


In [ ]:
# from datasets import load_dataset, concatenate_datasets

# ds_chat_sample=15000
# ds_code_sample=18000
# ds_math_sample=5000
# ds_daily_sample=5000
# # --- 1. SOHBET VERİSİ (Örn: UltraChat) ---
# # Orijinal sütunları: 'prompt' ve 'answers' diyelim.
# ds_chat = load_dataset("HuggingFaceH4/ultrachat_200k", split=f"train_sft[:{ds_chat_sample}]")
# ds_chat = ds_chat.map(lambda x: {
#     "instruction": x["prompt"], 
#     "output": x["messages"][1]["content"] # Veri yapısına göre seçiyoruz
# }, remove_columns=ds_chat.column_names)
# print("ds_chat is okay")

# # --- 2. KOD VERİSİ (Örn: Alpaca Code) ---
# # Orijinal sütunları: 'instruction', 'input', 'output'
# ds_code = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split=f"train[:{ds_code_sample}]")
# ds_code = ds_code.map(lambda x: {
#     "instruction": x["instruction"] + ("\n" + x["input"] if x["input"] else ""),
#     "output": x["output"]
# }, remove_columns=ds_code.column_names)
# print("ds_code is okay")
# # --- 3. MATEMATİK VERİSİ (Örn: GSM8K) ---
# # Orijinal sütunları: 'question', 'answer'
# ds_math = load_dataset("gsm8k", "main", split=f"train[:{ds_math_sample}]")
# ds_math = ds_math.map(lambda x: {
#     "instruction": x["question"],
#     "output": x["answer"]
# }, remove_columns=ds_math.column_names)
# print("ds_math is okay")

# ds_daily = load_dataset("daily_dialog", split=f"train[:{ds_daily_sample}]")

# # DailyDialog'da mesajlar liste halindedir. 
# # En basit yöntem: İlk mesajı 'instruction', ikinciyi 'output' yapmak.
# def format_daily(example):
#     # Eğer diyalog en az 2 mesaj içeriyorsa formatla
#     if len(example["dialog"]) >= 2:
#         return {
#             "instruction": example["dialog"][0],
#             "output": example["dialog"][1]
#         }
#     else:
#         # Boş veya tek mesajlıları temizlemek için işaretle
#         return {"instruction": None, "output": None}

# ds_daily = ds_daily.map(format_daily)
# # None olanları (tek mesajlı diyalogları) temizle ve sütunları eşitle
# ds_daily = ds_daily.filter(lambda x: x["instruction"] is not None)
# ds_daily = ds_daily.remove_columns([col for col in ds_daily.column_names if col not in ["instruction", "output"]])

# print("ds_daily is okay")
# # --- ŞİMDİ BİRLEŞTİREBİLİRİZ ---
# # Artık hepsinin sadece 2 sütunu var: 'instruction' ve 'output'
# dataset = concatenate_datasets([ds_chat, ds_code, ds_math,wiki_ds,ds_daily,])
# # Veri setlerini tek bir havuzda birleştirme
# dataset = dataset.shuffle()

ds_chat is okay
ds_code is okay
ds_math is okay


In [44]:
from datasets import load_dataset, concatenate_datasets

ds_chat_sample=11000
ds_code_sample=10000
ds_math_sample=4000
ds_dolly_sample=10000

# --- 1. SOHBET VERİSİ (Örn: UltraChat) ---
# Orijinal sütunları: 'prompt' ve 'answers' diyelim.
ds_chat = load_dataset("HuggingFaceH4/ultrachat_200k", split=f"train_sft[:{ds_chat_sample}]")
ds_chat = ds_chat.map(lambda x: {
    "instruction": x["prompt"], 
    "output": x["messages"][1]["content"] # Veri yapısına göre seçiyoruz
}, remove_columns=ds_chat.column_names)
ds_chat_split = ds_chat.train_test_split(test_size=0.05, seed=42)
train_ds_chat = ds_chat_split["train"]
val_ds_chat = ds_chat_split["test"]
print("ds_chat is okay")

# --- 2. KOD VERİSİ (Örn: Alpaca Code) ---
# Orijinal sütunları: 'instruction', 'input', 'output'
ds_code = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split=f"train[:{ds_code_sample}]")
ds_code = ds_code.map(lambda x: {
    "instruction": x["instruction"] + ("\n" + x["input"] if x["input"] else ""),
    "output": x["output"]
}, remove_columns=ds_code.column_names)
ds_code_split = ds_code.train_test_split(test_size=0.05, seed=42)
train_ds_code = ds_code_split["train"]
val_ds_code = ds_code_split["test"]
print("ds_code is okay")

# --- 3. MATEMATİK VERİSİ (Örn: GSM8K) ---
# Orijinal sütunları: 'question', 'answer'
ds_math = load_dataset("gsm8k", "main", split=f"train[:{ds_math_sample}]")
ds_math = ds_math.map(lambda x: {
    "instruction": x["question"],
    "output": x["answer"]
}, remove_columns=ds_math.column_names)
ds_math_split = ds_math.train_test_split(test_size=0.05, seed=42)
train_ds_math = ds_math_split["train"]
val_ds_math = ds_math_split["test"]
print("ds_math is okay")


# --- 6. DOLLY EKLEME ---
ds_dolly = load_dataset("databricks/databricks-dolly-15k", split=f"train[:{ds_dolly_sample}]")

def format_dolly(example):
    # 'context' varsa onu da instruction'a ekleyelim ki model bilgi kullanmayı öğrensin
    instruction = example["instruction"]
    if example.get("context"):
        instruction += "\nContext: " + example["context"]
    
    return {
        "instruction": instruction,
        "output": example["response"]
    }

ds_dolly = ds_dolly.map(format_dolly, remove_columns=ds_dolly.column_names)
ds_dolly_split = ds_dolly.train_test_split(test_size=0.05, seed=42)
train_ds_dolly = ds_dolly_split["train"]
val_ds_dolly = ds_dolly_split["test"]
print(f"Dolly hazır! {len(ds_dolly)} örnek eklendi.")


# --- ŞİMDİ BİRLEŞTİREBİLİRİZ ---
# Artık hepsinin sadece 2 sütunu var: 'instruction' ve 'output'
dataset = concatenate_datasets([train_ds_chat, train_ds_code, train_ds_math, train_wiki_ds, train_ds_daily,train_ds_dolly])

# Veri setlerini tek bir havuzda birleştirme
dataset = dataset.shuffle()

val_dataset_dict = {
    "chat": val_ds_chat,
    "code": val_ds_code,
    "math": val_ds_math,
    "daily": val_ds_daily,
    "wiki": val_wiki_ds,
    "dolly": val_ds_dolly
}

ds_chat is okay
ds_code is okay
ds_math is okay


Map: 100%|██████████| 10000/10000 [00:00<00:00, 25289.26 examples/s]


Dolly hazır! 10000 örnek eklendi.


In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "gpt2" # veya "gpt2-medium", "gpt2-large"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# # GPT-2'de pad_token tanımlı değildir, eos_token'ı pad_token olarak atıyoruz
# tokenizer.pad_token = tokenizer.eos_token 

# Veri setini GPT-2'nin anlayacağı formata sokma
def format_for_gpt2(example):
    # Soru ve cevabı özel belirteçlerle birleştiriyoruz
    # GPT-2 için genellikle <|endoftext|> kullanılır ama araya ayıraç koymak iyidir
    full_text = f"Question: {example['instruction']}\nAnswer: {example['output']}{tokenizer.eos_token}"
    return {"text": full_text}

# Bir önceki adımda oluşturduğun combined_dataset'i kullanıyoruz
gpt2_dataset = dataset.map(format_for_gpt2, remove_columns=dataset.column_names)

Map: 100%|██████████| 52250/52250 [00:04<00:00, 11930.45 examples/s]


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

mpath="./gpt2-multitask-model-new/checkpoint-818"
mpath="gpt2"
model = AutoModelForCausalLM.from_pretrained(mpath)
tokenizer = AutoTokenizer.from_pretrained(mpath)
# Veriyi tokenlara ayırma (Tokenization)
def tokenize_function(examples):
    tok = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tok["labels"] = tok["input_ids"].copy()  # labels ekleniyor
    return tok

# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")
#gpt2_dataset = gpt2_dataset.shuffle()
tokenized_dataset = gpt2_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Tüm validasyon setlerini önce format et, sonra tokenize et
tokenized_vals = {
    name: ds.map(format_for_gpt2, remove_columns=ds.column_names).map(tokenize_function, batched=True, remove_columns=["text"]) 
    for name, ds in val_dataset_dict.items()
}

# Modeli yükleme
# model = AutoModelForCausalLM.from_pretrained(model_id)

# Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./gpt2-multitask-model-new",
    overwrite_output_dir=True,
    num_train_epochs=5,            # Veri üzerinden 3 tur geç
    per_device_train_batch_size=32, # GPU belleğine göre artırabilirsin
    save_steps=300,
    logging_steps=100,
    eval_steps=300,
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=3e-6,            # GPT-2 için ideal öğrenme oranı
    weight_decay=0.1,
    no_cuda=False,
    report_to="none",
    prediction_loss_only=True,
)



# Data Collator: Verileri modelin okuyabileceği paketler haline getirir
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_vals["chat"],  # İstersen tek bir validasyon seti ile başlayabilirsin
    data_collator=data_collator,
)

# Eğitimi başlat
trainer.train()

Map: 100%|██████████| 250/250 [00:00<00:00, 5082.48 examples/s]


Epoch,Training Loss,Validation Loss
1,2.042800,1.780606
2,2.044400,1.781915
3,2.046000,1.782603
4,2.044400,1.782934
5,2.042900,1.783075


TrainOutput(global_step=2045, training_loss=2.0441091374255045, metrics={'train_runtime': 3131.7632, 'train_samples_per_second': 83.419, 'train_steps_per_second': 0.653, 'total_flos': 6.826254336e+16, 'train_loss': 2.0441091374255045, 'epoch': 5.0})

In [46]:
def ask_gpt2(question):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda") # GPU varsa
    outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Test
ask_gpt2("Write a python function to sort a list.")
ask_gpt2("What is 2 + 2?")
ask_gpt2("Who is the Obama?")
ask_gpt2("About world war 2?")
ask_gpt2("write a short poem about AI.")

Question: Write a python function to sort a list.
Answer: def sort_list(lst):
    for i in range(len(lst)):
        min_idx = i
        for j in range(
Question: What is 2 + 2?
Answer: 2 + 2 is a number that is divisible by 3. Therefore, it is divisible by 3 / 3 = <<2/3=1>>1.
#### 1 is the number that is divisible by 3. Therefore, it is
Question: Who is the Obama?
Answer: The Obama is the first president elected by the American people. He is the first president elected by the American people and is the first to be elected by the popular vote. The president is elected by the people and is accountable to the people for their decisions
Question: About world war 2?
Answer: World war 2 was a conflict between the Soviet Union and Nazi Germany that lasted from 1939 to 1945. The conflict was fought between the Soviet Union and Nazi Germany over control of the Black Sea Fleet and the control of the Mediterranean Sea. The Soviet Union was
Question: write a short poem about AI.
Answer: In the wor